In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier,\
                             GradientBoostingClassifier, ExtraTreesClassifier,\
                             BaggingClassifier, VotingClassifier)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import KFold

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

from sklearn.metrics import roc_auc_score


from time import time

# Set de entrenamiento y Test final

Cargamos los archivos necesarios

In [67]:
train = pd.read_csv('train_final.csv')
test_final = pd.read_csv('test_final.csv')

In [78]:
train.person.count()

19414

In [79]:
test.person.count()

5181

Transformamos las variables categóricas al tipo correspondiente, para reducir el uso de memoria

In [3]:
#train['rango_edad'] = train['rango_edad'].astype('category')
#train['sexo'] = train['sexo'].astype('category') 
#train['nivel_estudios'] = train['nivel_estudios'].astype('category')
#train['esta_estudiando'] = train['esta_estudiando'].astype('category')
#train['tipo_de_trabajo'] = train['tipo_de_trabajo'].astype('category')
#train['nivel_laboral'] = train['nivel_laboral'].astype('category')
#train['nombre_zona'] = train['nombre_zona'].astype('category')
#train['num_area'] = train['num_area'].astype('category')
#train['post_por_nivel'] = train['post_por_nivel'].astype('category')
#train['sepostulo'] = train['sepostulo'].astype('category')

In [4]:
#test_final['rango_edad'] = test_final['rango_edad'].astype('category')
#test_final['sexo'] = test_final['sexo'].astype('category') 
#test_final['nivel_estudios'] = test_final['nivel_estudios'].astype('category')
#test_final['esta_estudiando'] = test_final['esta_estudiando'].astype('category')
#test_final['tipo_de_trabajo'] = test_final['tipo_de_trabajo'].astype('category')
#test_final['nivel_laboral'] = test_final['nivel_laboral'].astype('category')
#test_final['nombre_zona'] = test_final['nombre_zona'].astype('category')
#test_final['num_area'] = test_final['num_area'].astype('category')
#test_final['post_por_nivel'] = test_final['post_por_nivel'].astype('category')

Nos guardamos los IDs necesarios para el submit

In [5]:
#id_aviso_postulante = test_final['id']

Analizamos el set de entrenamiento y el test final

In [41]:
train.sample(5)

,person,brand listing,checkout,conversion,generic listing,searched products,staticpage,viewed product,visited site,label
380,64a3c2f6,14.0,0.0,0.0,1.0,0.0,0.0,13.0,7.0,1.0
705,b866afff,39.0,4.0,2.0,28.0,101.0,2.0,78.0,20.0,1.0
1474,947126c0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0
160,2b81e8f3,10.0,1.0,1.0,19.0,0.0,0.0,115.0,9.0,1.0
1321,f4924234,1.0,2.0,0.0,0.0,0.0,0.0,20.0,5.0,0.0


In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1980 entries, 0 to 1979
Data columns (total 10 columns):
person               1980 non-null object
brand listing        1980 non-null float64
checkout             1980 non-null float64
conversion           1980 non-null float64
generic listing      1980 non-null float64
searched products    1980 non-null float64
staticpage           1980 non-null float64
viewed product       1980 non-null float64
visited site         1980 non-null float64
label                1980 non-null float64
dtypes: float64(9), object(1)
memory usage: 154.8+ KB


In [28]:
test_final.sample(5)

NameError: name 'test_final' is not defined

In [5]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19414 entries, 0 to 19413
Data columns (total 10 columns):
person               19414 non-null object
label                19414 non-null int64
brand listing        19414 non-null float64
checkout             19414 non-null float64
conversion           19414 non-null float64
generic listing      19414 non-null float64
searched products    19414 non-null float64
staticpage           19414 non-null float64
viewed product       19414 non-null float64
visited site         19414 non-null float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.5+ MB


Nos guardamos los features que utilizaremos para los algoritmos de Machine Learning

In [68]:
features = train.iloc[:,2:].columns
features

Index(['compras_1', 'compras_2', 'compras_3', 'compras_4', 'compras_5',
       'prom_tiempo_1', 'prom_tiempo_2', 'prom_tiempo_3', 'prom_tiempo_4',
       'prom_tiempo_5', 'ad campaign hit', 'brand listing', 'checkout',
       'conversion', 'generic listing', 'lead', 'search engine hit',
       'searched products', 'staticpage', 'viewed product', 'visited site',
       'antiguedad', 'max_sin_convertir', 'dias_sin_convertir',
       'convierte_cada_x_dias'],
      dtype='object')

In [30]:
# Features
f1 = 'ad campaign hit'
f2 = 'brand listing'
f3 = 'checkout'
f4 = 'conversion'
f5 = 'generic listing'
f6 = 'search engine hit'
f7 = 'viewed product'
f8 = 'visited site'
f9 = 'searched products'
f10= 'staticpage'

features = [f2,f3,f4,f5,f9,f10,f7,f8]

Particionamos el set de entrenamiento para realizar pruebas locales de hiper-parámetros antes de realizar cada submit

In [69]:
test_s = 0.25
random_s = 0

x = np.array(train[features])
y = np.array(train['label'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_s, random_state=random_s)

Generamos el set de entrenamiento para el test final

In [70]:
x_test_final = np.array(test_final[features])

# Algoritmos probados

# 1. KNN

### Valores iniciales para K

In [71]:
t0 = time()

print('VALORES INICIALES PARA K')

k_valores = [3,5,10,20,50,60,75,80]
mejor_k = 0
mejor_precision = 0

for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = roc_auc_score(y_test, pred)
    print('La precisión para k=', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

VALORES INICIALES PARA K
La precisión para k= 3 es: 0.5174983745123538
La precisión para k= 5 es: 0.5065994798439531
La precisión para k= 10 es: 0.5020833333333333
La precisión para k= 20 es: 0.5
La precisión para k= 50 es: 0.5
La precisión para k= 60 es: 0.5
La precisión para k= 75 es: 0.5
La precisión para k= 80 es: 0.5
Con k= 3 se obtuvo la mayor precisión: 0.5174983745123538
Tiempo de ejecución: 9.16987 seconds.


In [72]:
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    print(pd.Series(y_test).value_counts())
    print (pd.Series(pred).value_counts())

0    4614
1     240
dtype: int64
0    4624
1     230
dtype: int64


### Valores grid search para K

In [73]:
t0 = time()

print('VALORES GRID SEARCH PARA K')

k_valores = []

for i in range(mejor_k-2, mejor_k+3):
    k_valores.append(i)
    
for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = roc_auc_score(y_test, pred)
    print('La precisión para k =', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

VALORES GRID SEARCH PARA K
La precisión para k = 1 es: 0.5386351322063286
La precisión para k = 2 es: 0.5068162115301257
La precisión para k = 3 es: 0.5174983745123538
La precisión para k = 4 es: 0.5057081707845688
La precisión para k = 5 es: 0.5065994798439531
Con k= 1 se obtuvo la mayor precisión: 0.5386351322063286
Tiempo de ejecución: 5.31066 seconds.


### Predecimos el test final con el mejor k obtenido

In [57]:
# Creamos el KNN
knn = KNeighborsClassifier(n_neighbors=1)

# Lo entrenamos con la totalidad del set de datos
knn.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = knn.predict(x_test_final)

### Submit File

In [60]:
person = test_final['person']

submit = pd.DataFrame({'person':person, 'label':pred_final})

submit.to_csv('submit_knn.csv', index=False)

In [62]:
submit.groupby('person').count().label.value_counts()

3    700
2    531
1    527
4    367
6      2
7      1
5      1
Name: label, dtype: int64

# 2. Random Forest

### Set de entrenamiento

In [74]:
t0 = time()

# Parámetros óptimos
params_optimos_rf = {'n_estimators':50, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el Random Forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con nuestro set de entrenamiento
random_forest.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = random_forest.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5020833333333333
Tiempo de ejecución: 0.49791 seconds.


### Test final

#### Predicciones con valores binarios

In [10]:
t0 = time()

# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = random_forest.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

NameError: name 'x_test_final' is not defined

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el random forest
random_forest = RandomForestClassifier(**params_optimos_rf)

# Lo entrenamos con la totalidad del set de datos
random_forest.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = random_forest.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit files

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_random_forest.csv', index=False)

In [ ]:
#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_random_forest_proba.csv', index=False)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")
roc_auc_score(np.array(test_final['label']),pred_final_proba)

# 3. Extra Trees

### Set de entrenamiento

In [75]:
t0 = time()

# Parámetros óptimos
params_optimos_et = {'n_estimators':100, 'max_features':'sqrt', 'max_depth':5, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'bootstrap':True, 'oob_score':True, 'warm_start':True}

# Creamos el extra_trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con nuestro set de entrenamiento
extra_trees.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = extra_trees.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5
Tiempo de ejecución: 0.43183 seconds.


### Test final

#### Predicciones con valores binarios

In [9]:
t0 = time()

# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = extra_trees.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

NameError: name 'x_test_final' is not defined

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el extra trees
extra_trees = ExtraTreesClassifier(**params_optimos_et)

# Lo entrenamos con la totalidad del set de datos
extra_trees.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = extra_trees.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_extra_trees.csv', index=False)

In [ ]:
#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_extra_trees_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 4. Gaussian Naive Bayes

### Set de entrenamiento

In [76]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con nuestro set de entrenamiento
naive_bayes.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = naive_bayes.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5782347204161248
Tiempo de ejecución: 0.09366 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = naive_bayes.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el naive bayes
naive_bayes = GaussianNB()

# Lo entrenamos con la totalidad del set de datos
naive_bayes.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = naive_bayes.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_naive_bayes.csv', index=False)

In [ ]:
#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_naive_bayes_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 5. Perceptron

### Set de entrenamiento

In [12]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con nuestro set de entrenamiento
perceptron.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = perceptron.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.508573717948718
Tiempo de ejecución: 0.00304 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el perceptron
perceptron = Perceptron()

# Lo entrenamos con la totalidad del set de datos
perceptron.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = perceptron.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_perceptron.csv', index=False)

In [ ]:
#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

# 6. Adaptive Boosting

### Set de entrenamiento

In [13]:
t0 = time()

# Parámetros óptimos
params_optimos_ab = {'n_estimators':50}

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con nuestro set de entrenamiento
ada_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = ada_boost.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5150641025641026
Tiempo de ejecución: 0.16708 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = ada_boost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el adaptive boosting
ada_boost = AdaBoostClassifier(**params_optimos_ab)

# Lo entrenamos con la totalidad del set de datos
ada_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = ada_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_ada_boost.csv', index=False)

In [ ]:
#submit['sepostulo'].value_counts()
roc_auc_score(np.array(test_final['label']),pred_final)

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_ada_boost_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 7. Gradient Boosting

### Set de entrenamiento

In [14]:
t0 = time()

# Parámetros óptimos
params_optimos_gb = {'learning_rate':0.1, 'n_estimators':150, 'max_depth':4, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'subsample':0.5, 'max_features':'sqrt', 'warm_start':True}

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con nuestro set de entrenamiento
gra_boost.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = gra_boost.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5140224358974359
Tiempo de ejecución: 0.29785 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = gra_boost.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el gradient boosting
gra_boost = GradientBoostingClassifier(**params_optimos_gb)

# Lo entrenamos con la totalidad del set de datos
gra_boost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = gra_boost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_gra_boost.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

In [ ]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_gra_boost_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 8. Logistic Regression

### Set de entrenamiento

In [15]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con nuestro set de entrenamiento
log_reg.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = log_reg.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5278044871794872
Tiempo de ejecución: 0.11773 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = log_reg.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el logistic regression
log_reg = LogisticRegression()

# Lo entrenamos con la totalidad del set de datos
log_reg.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = log_reg.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_log_reg.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

In [ ]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_log_reg_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 9. Decision Tree

### Set de entrenamiento

In [16]:
t0 = time()

# Parámetros óptimos
params_optimos_dt = {'criterion':'entropy', 'splitter':'best', 'max_depth':None, 'min_samples_split':2,\
                     'min_samples_leaf':2, 'max_features':'sqrt', 'presort':True}

# Creamos el logistic regression
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con nuestro set de entrenamiento
decision_tree.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = decision_tree.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5124198717948718
Tiempo de ejecución: 0.00901 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = decision_tree.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el decision tree
decision_tree = DecisionTreeClassifier(**params_optimos_dt)

# Lo entrenamos con la totalidad del set de datos
decision_tree.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = decision_tree.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_decision_tree.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

In [ ]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_decision_tree_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 10. Bagging

## 10.1 Con Logistic Regression

### Set de entrenamiento

In [17]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train,y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.49895833333333334
Tiempo de ejecución: 6.05670 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

params_optimos_bag_lr = {'base_estimator':LogisticRegression(), 'n_estimators':100, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_lr)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = bagging.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_bagging_log_reg.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

In [ ]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_bagging_log_reg_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

## 10.2 Con Decision Tree

### Set de entrenamiento

In [18]:
t0 = time()

# Parámetros óptimos
params_optimos_bag_dt = {'base_estimator':DecisionTreeClassifier(**params_optimos_dt), 'n_estimators':10, 'bootstrap':True,\
                         'bootstrap_features':True, 'oob_score':True, 'warm_start':False}

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con nuestro set de entrenamiento
bagging.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = bagging.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5075320512820513
Tiempo de ejecución: 0.07600 seconds.


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = bagging.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el bagging
bagging = BaggingClassifier(**params_optimos_bag_dt)

# Lo entrenamos con la totalidad del set de datos
bagging.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = bagging.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_bagging_decision_tree.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

In [ ]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_bagging_decision_tree_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")

# 11. Voting

### Set de entrenamiento

In [19]:
# Creamos los estimadores base que utilizaremos para el voting
estimador_1 = DecisionTreeClassifier(**params_optimos_dt)
estimador_2 = GradientBoostingClassifier(**params_optimos_gb)
estimador_3 = ExtraTreesClassifier(**params_optimos_et)

# Los agregamos a la lista de estimadores
estimadores = []
estimadores.append(('Naive Bayes', estimador_1))
estimadores.append(('Logistic Regression', estimador_2))
estimadores.append(('Random Forest', estimador_3))

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con nuestro set de entrenamiento
voting.fit(x_train, y_train)

# Predecimos valores de nuestro set de datos
pred = voting.predict(x_test)

# Evaluamos la precisión
precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5075320512820513
Tiempo de ejecución: 8.37632 seconds.


/home/mariano/ENTER/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Test final

#### Predicciones con valores binarios

In [ ]:
t0 = time()

# Creamos el voting
voting = VotingClassifier(estimators=estimadores)

# Lo entrenamos con la totalidad del set de datos
voting.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = voting.predict(x_test_final)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_voting.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

In [ ]:
#submit['sepostulo'].value_counts()

# 12. XGBoost

### Set de entrenamiento

In [77]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold

t0 = time()
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

folds = 2
param_comb = 150

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgboost, param_distributions=params, \
                                   n_iter=param_comb, scoring='roc_auc', \
                                   n_jobs=4, cv=skf.split(x,y), verbose=3,\
                                   random_state=1001 )

random_search.fit(x,y)

print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Fitting 2 folds for each of 150 candidates, totalling 300 fits
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=0.6, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=0.8 
[CV] subsample=0.6, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=0.8 
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=0.8349776256385162, total=   4.5s
[CV] subsample=0.8, min_child_weight=5, max_depth=5, gamma=1, colsample_bytree=0.8 
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=0.8513673712948346, total=   4.6s
[CV] subsample=0.8, min_child_weight=5, max_depth=5, gamma=1, colsample_bytree=0.8 
[CV]  subsample=0.6, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=0.8, score=0.8400580781298088, total=   7.7s
[CV] subsample=1.0, min_child_weight=5, max_depth=5,

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   37.3s


[CV]  subsample=0.6, min_child_weight=10, max_depth=5, gamma=1, colsample_bytree=0.6, score=0.8395964201021626, total=   7.1s
[CV] subsample=1.0, min_child_weight=1, max_depth=5, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=0.6, min_child_weight=10, max_depth=5, gamma=1, colsample_bytree=0.6, score=0.8494919547508707, total=   7.3s
[CV] subsample=1.0, min_child_weight=1, max_depth=5, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0, score=0.8452092739015972, total=   8.2s
[CV] subsample=0.8, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.8 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0, score=0.8374540832932935, total=   8.4s
[CV] subsample=0.8, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.8 
[CV]  subsample=1.0, min_child_weight=1, max_depth=5, gamma=0.5, colsample_bytree=1.0, score=0.83783647341979, total=   7.9s
[CV] subsample=1.0, min

[CV]  subsample=1.0, min_child_weight=10, max_depth=5, gamma=1.5, colsample_bytree=0.8, score=0.8423852995684549, total=   5.3s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=1.5, colsample_bytree=0.8, score=0.849761089202959, total=   6.0s
[CV] subsample=0.8, min_child_weight=10, max_depth=5, gamma=2, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=10, max_depth=5, gamma=1.5, colsample_bytree=0.8, score=0.8515240250380287, total=   5.1s
[CV] subsample=0.8, min_child_weight=10, max_depth=5, gamma=2, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=1.0, score=0.8375573751253783, total=   8.0s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=1, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=5, colsample_bytree=1.0, score=0.851790945302934, total=   8.0s
[CV] subsample=1.0, min_child_w

[CV]  subsample=0.6, min_child_weight=5, max_depth=4, gamma=1, colsample_bytree=0.6, score=0.8402676509466759, total=   6.7s
[CV] subsample=0.6, min_child_weight=10, max_depth=4, gamma=1, colsample_bytree=0.8 
[CV]  subsample=0.8, min_child_weight=10, max_depth=3, gamma=1.5, colsample_bytree=1.0, score=0.8405496276844252, total=   7.1s
[CV] subsample=0.6, min_child_weight=10, max_depth=4, gamma=1, colsample_bytree=0.8 
[CV]  subsample=0.6, min_child_weight=5, max_depth=4, gamma=1, colsample_bytree=0.6, score=0.8495214034404039, total=   7.3s
[CV] subsample=1.0, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV]  subsample=0.8, min_child_weight=10, max_depth=3, gamma=1.5, colsample_bytree=1.0, score=0.8525149845117607, total=   7.3s
[CV] subsample=1.0, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=10, max_depth=5, gamma=5, colsample_bytree=0.6, score=0.8368916797488226, total=   5.9s
[CV] subsample=0.8, min_chil

...

[CV]  subsample=1.0, min_child_weight=10, max_depth=4, gamma=1, colsample_bytree=0.6, score=0.8524923998024945, total=   4.4s
[CV] subsample=1.0, min_child_weight=10, max_depth=3, gamma=1, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=10, max_depth=4, gamma=1, colsample_bytree=0.6, score=0.8411979416915947, total=   4.5s
[CV] subsample=1.0, min_child_weight=10, max_depth=3, gamma=1, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=1, max_depth=3, gamma=0.5, colsample_bytree=1.0, score=0.8393869579946549, total=   6.1s
[CV] subsample=0.8, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=1, max_depth=3, gamma=0.5, colsample_bytree=1.0, score=0.8500161635664356, total=   6.3s
[CV] subsample=0.8, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.6 


[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:  8.0min


[CV]  subsample=1.0, min_child_weight=10, max_depth=3, gamma=1, colsample_bytree=0.6, score=0.8523375174090467, total=   4.4s
[CV] subsample=0.8, min_child_weight=10, max_depth=3, gamma=1.5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=10, max_depth=3, gamma=1, colsample_bytree=0.6, score=0.840492612364464, total=   4.5s
[CV] subsample=0.8, min_child_weight=10, max_depth=3, gamma=1.5, colsample_bytree=0.6 
[CV]  subsample=0.8, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.6, score=0.840402605655477, total=   5.3s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=2, colsample_bytree=0.8 
[CV]  subsample=0.8, min_child_weight=5, max_depth=3, gamma=2, colsample_bytree=0.6, score=0.8516936317762431, total=   5.2s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=2, colsample_bytree=0.8 
[CV]  subsample=0.8, min_child_weight=10, max_depth=3, gamma=1.5, colsample_bytree=0.6, score=0.839896331756094, total=   4.7s
[CV] subsample=1.0, min_child_w

[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:  8.6min finished



 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=2, learning_rate=0.02, max_delta_step=0,
       max_depth=4, min_child_weight=10, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1.0)

 Best normalized gini score for 2-fold search with 150 parameter combinations:
0.6969409896973873

 Best hyperparameters:
{'subsample': 1.0, 'min_child_weight': 10, 'max_depth': 4, 'gamma': 2, 'colsample_bytree': 0.8}
Tiempo de ejecución: 522.88313 seconds.


In [ ]:
pred = random_search.predict_proba(test)

In [26]:
print(random_search.cv_results_)


{'mean_fit_time': array([0.60764583, 1.05604895, 1.07023795, 0.67376773, 0.99590214]), 'std_fit_time': array([0.00907994, 0.06183087, 0.05643955, 0.04548575, 0.05476531]), 'mean_score_time': array([0.0042158 , 0.02445277, 0.0250682 , 0.00460092, 0.02046879]), 'std_score_time': array([0.00064934, 0.00270637, 0.00106131, 0.00038284, 0.00019438]), 'param_subsample': masked_array(data=[1.0, 0.6, 0.8, 1.0, 0.8],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 1, 5, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 5, 5, 5, 4],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[5, 1.5, 1, 5, 1],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'p

### Test final

#### Predicciones con valores binarios

In [27]:
t0 = time()

# Creamos el xgboost

xgboost= xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,\
       colsample_bytree=0.6, gamma=5, learning_rate=0.02, max_delta_step=0,\
       max_depth=5, min_child_weight=5, missing=None, n_estimators=600,\
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,\
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,\
       silent=True, subsample=1.0)

# Lo entrenamos con la totalidad del set de datos
xgboost.fit(x_train,y_train)

# Predecimos las postulaciones del set final
pred = xgboost.predict(x_test)

precision = roc_auc_score(y_test, pred)

print("Precisión: ", precision)

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

Precisión:  0.5
Tiempo de ejecución: 0.57988 seconds.


/home/mariano/ENTER/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Predicciones con probabilidades

In [ ]:
t0 = time()

# Creamos el xgboost
xgboost = xgb.XGBClassifier(learning_rate = 0.02,\
                            n_estimators= 2000,\
                            max_depth= 5,\
                            min_child_weight= 2,\
                            gamma=0.9,\
                            colsample_bytree=0.7,\
                            objective= 'binary:logistic',\
                            nthread= -1,\
                            scale_pos_weight=1)

# Lo entrenamos con la totalidad del set de datos
xgboost.fit(x,y)

# Predecimos las postulaciones del set final
pred_final_proba = xgboost.predict_proba(x_test_final)

# Nos quedamos con la columna correspondiente de probabilidades
df_predicciones = pd.DataFrame(pred_final_proba)
pred_final_proba = np.array(df_predicciones[1])

tf = time() - t0
print ("Tiempo de ejecución: %0.5f seconds." % tf)

### Submit file

#### Submit con valores binarios

In [ ]:
#submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})
#submit.to_csv('submits/submit_xgboost.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final)

In [ ]:
#submit['sepostulo'].value_counts()

#### Submit con probabilidades

In [ ]:
#submit_proba = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final_proba})
#submit_proba.to_csv('submits/submit_xgboost_proba.csv', index=False)
roc_auc_score(np.array(test_final['label']),pred_final_proba)

In [ ]:
#no = submit_proba['sepostulo'] < 0.5
#si = submit_proba['sepostulo'] >= 0.5

#cant_no = submit_proba.loc[(no)].count()
#cant_si = submit_proba.loc[(si)].count()

#print("0   ", cant_no[1])
#print("1   ", cant_si[1])
#print("Name: sepostulo, dtype: int64")